In [1]:
import torch
torch.random.manual_seed(0)

import linear_operator

# Numerics and computing
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("DEVICE = ", DEVICE)

DEVICE =  cpu


## Problem definition

In [2]:
N = 6

# Operators `K` and `Winv`
K = 2 * torch.rand(N, N) - 1
K = K @ K.T + 1e-6 * torch.eye(N)
K = K.to(DEVICE)
print("K = \n", K)

Winv = torch.diag(2 * torch.rand(N) - 1)
print("Winv = \n", Winv)

# Linear system solution
x = 2 * torch.rand(N, device=DEVICE) - 1

# Linear system right hand side
rhs = (K + Winv) @ x

K = 
 tensor([[ 1.7271e+00,  3.6748e-01, -2.7473e-02, -8.5865e-01,  1.7001e-01,
          1.3566e+00],
        [ 3.6748e-01,  8.3693e-01, -6.9722e-01, -2.2924e-01,  1.2817e-03,
          1.0237e+00],
        [-2.7473e-02, -6.9722e-01,  2.0389e+00,  1.1847e+00, -7.2450e-01,
         -7.8783e-01],
        [-8.5865e-01, -2.2924e-01,  1.1847e+00,  2.0730e+00, -4.3813e-01,
         -1.2867e+00],
        [ 1.7001e-01,  1.2817e-03, -7.2450e-01, -4.3813e-01,  2.1780e+00,
          6.7130e-01],
        [ 1.3566e+00,  1.0237e+00, -7.8783e-01, -1.2867e+00,  6.7130e-01,
          2.8957e+00]])
Winv = 
 tensor([[-0.5837,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.8596,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.4462,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.4847,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0526,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.5127]])


## First solve

For the first solve, we restrict the number of iterations to `3`.

In [3]:
from linear_operator.linear_solvers import PLS_GPC
from linear_operator.linear_solvers.policies import UnitVectorPolicy, GradientPolicy


pls = PLS_GPC(policy=GradientPolicy(), max_iter=3)

with torch.no_grad():
    for solver_state in pls.solve_iterator(K, Winv, rhs=rhs):
        print("\nIteration", solver_state.iteration)
        print("x =", solver_state.solution)
        print("actions = \n", solver_state.cache["actions"])
        print("K_op_actions = \n", solver_state.cache["K_op_actions"])

[KeOps] Warning : 
    The default C++ compiler could not be found on your system.
    You need to either define the CXX environment variable or a symlink to the g++ command.
    For example if g++-8 is the command you can do
      import os
      os.environ['CXX'] = 'g++-8'
    
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode
[PLS_GPC] Case 2: Setting `x` to zero

Iteration 0
x = tensor([0., 0., 0., 0., 0., 0.])
actions = 
 None
K_op_actions = 
 None

Iteration 1
x = tensor([ 0.2351, -0.0578, -0.5181, -0.6066,  0.4985,  0.4500])
actions = 
 tensor([[ 1.2155],
        [-0.2989],
        [-2.6791],
        [-3.1367],
        [ 2.5779],
        [ 2.3271]])
K_op_actions = 
 tensor([[  8.3516],
        [  5.1691],
        [-12.7049],
        [-14.7756],
        [ 10.6986],
        [ 15.9589]])

Iteration 2
x = tensor([ 0.1553, -0.6285, -0.5982, -0.6985,  0.7965,  0.2856])
actions = 
 tensor([[ 1.2155, -0.2623],
        [-0.2989, -1.2488],
        [-2.

## Second (preconditioned) solve

Now, we let the solver run for a maximum of `N = 6` iterations. Note that we use the same `Winv` for the second run. So, the solver converges in `3` iterations (because it has already performed `3` in the first run).

In [4]:
actions = solver_state.cache["actions"]
K_op_actions = solver_state.cache["K_op_actions"]

pls = PLS_GPC(policy=GradientPolicy(), max_iter=N)

with torch.no_grad():
    solve_iterator = pls.solve_iterator(
        K, 
        Winv, 
        rhs, 
        actions=actions, 
        K_op_actions=K_op_actions
    )
    
    for solver_state in solve_iterator:
        print("\nIteration", solver_state.iteration)
        print("x =", solver_state.solution)
        print("actions = \n", solver_state.cache["actions"])
        print("K_op_actions = \n", solver_state.cache["K_op_actions"])

[PLS_GPC] Case 1: Preconditioning

Iteration 0
x = tensor([ 0.2711, -0.8387, -0.6032, -0.6463,  0.6781,  0.4354])
actions = 
 None
K_op_actions = 
 None

Iteration 1
x = tensor([ 0.2790, -0.7570, -0.4594, -0.7846,  0.7216,  0.3549])
actions = 
 tensor([[ 0.0638],
        [-0.0091],
        [ 0.1574],
        [-0.1289],
        [-0.0079],
        [-0.0182]])
K_op_actions = 
 tensor([[ 0.1871],
        [-0.0831],
        [ 0.1930],
        [-0.1065],
        [-0.0762],
        [ 0.0609]])

Iteration 2
x = tensor([ 4.9779, -0.2487, -3.8829, -3.3301, -2.5527, -5.8756])
actions = 
 tensor([[ 0.0638, -0.1158],
        [-0.0091,  0.0184],
        [ 0.1574, -0.3255],
        [-0.1289,  0.2533],
        [-0.0079,  0.0070],
        [-0.0182,  0.0218]])
K_op_actions = 
 tensor([[ 0.1871, -0.3710],
        [-0.0831,  0.1641],
        [ 0.1930, -0.3955],
        [-0.1065,  0.2035],
        [-0.0762,  0.1350],
        [ 0.0609, -0.1399]])

Iteration 3
x = tensor([ 57.0261,   7.3530, -12.9444,   6.65